In [1]:
import pandas as pd

# User-Dataframe

In [7]:
import os

folder_path = './data'
list_of_filenames = os.listdir(folder_path)

In [30]:
info_files = []
tracking_files = []

for file in list_of_filenames :
    
    if file.split("_")[1] == "info":
        info_files.append(file)
    else:
        tracking_files.append(file)

In [35]:
users_list = []

for file in info_files:
    
    with open('./data/' + file, 'r') as file:
        info = file.read()
    
    if info.split(", ")[1] != "999": #used as testing indicator
        users_list.append(info.split(", "))

users = pd.DataFrame(users_list, columns=["uuid", "age", "gender", "device", "sorting_option"])
users

,uuid,age,gender,device,sorting_option
0,649aba058c612604f08fde63,23,female,touchpad,3
1,64995bac8c61d60b08d58991,26,female,mouse,1


# filenames verkürzen und in neuen Ordner

In [65]:
for file in tracking_files:
    
    new_file = file.split("_")[0] + "_" + file.split("_")[2]
    
    csv = pd.read_csv("./data/" + file, names = ["action", "x", "y", "ms"], header = None, index_col = [0])
    csv.set_index("ms")
    csv.to_csv("./data/tracking_files/" + new_file + ".csv")

# User überprüfen

## Anzahl Files

In [42]:
num_of_files_per_user = []

for uuid in users["uuid"]:
    
    user_files = []
    
    for file in tracking_files:
        
        if file.split("_")[0] == uuid:
            user_files.append(file)
    
    num_of_files_per_user.append(len(user_files))

users["num_of_files"] = num_of_files_per_user

In [48]:
users

,uuid,age,gender,device,sorting_option,num_of_files
0,649aba058c612604f08fde63,23,female,touchpad,3,23
1,64995bac8c61d60b08d58991,26,female,mouse,1,23


## Koordinaten der Buttons

In [85]:
coords_match = []

for uuid in users["uuid"]:
    
    a1 = pd.read_csv("./data/tracking_files/" + uuid + "_A1.csv")
    
    x = a1.sort_values("ms").iloc[-1]["x"]
    y = a1.sort_values("ms").iloc[-1]["y"]
    
    if 1315 <= x <= 1381.8999938964844 and 745 <= y <= 783:
        coords_match.append(True)
    else:
        coords_match.append(False)

users["coords_match"] = coords_match

# Fehlerhafte User entfernen

In [86]:
users

,uuid,age,gender,device,sorting_option,num_of_files,coords_match
0,649aba058c612604f08fde63,23,female,touchpad,3,23,True
1,64995bac8c61d60b08d58991,26,female,mouse,1,23,True


In [47]:
users.drop((users["num_of_files"] != 23).index)

,uuid,age,gender,device,sorting_option,num_of_files


In [88]:
users.drop((users["coords_match"] == False).index)

,uuid,age,gender,device,sorting_option,num_of_files,coords_match
